In [1]:
import pandas as pd
from data_processor import DataProcessor as dp

In [35]:
# Force all columns to be read as strings to prevent automatic type conversion
fatture_categorizzate = pd.read_csv('categorized_output.csv', dtype=str)
print(f"Fatture categorizzate dataset shape: {fatture_categorizzate.shape}")
print(f"P_IVA column type: {fatture_categorizzate['P_IVA'].dtype}")
print(fatture_categorizzate.sample(20).to_string(index=False))

Fatture categorizzate dataset shape: (25194, 16)
P_IVA column type: object
                      SourceName CODICE_COMMITTENTE ALIQUOTA_IVA       DATA       P_IVA UNITAMISURA IMPORTO_TOTALE_DOCUMENTO     NUMERO TIPO_DOCUMENTO                          RAGIONE_SOCIALE       SEDE_INDIRIZZO_CESSIONARIO    QUANTITA                                     DESCRIZIONE IMPONIBILE_IMPORTO PREZZO_UNITARIO               CATEGORIA
             IT04157540966_4btx4        10092750156          4.0 2024-03-20 02827030962         NaN                   380.73      4btx4           TD01                      METRO Italia S.p.A.          VIA FRANCESCO ALGAROTTI        2.00                           G500 STRACCIATELLA MC              11.98            5.99                    Food
             IT01533080675_6sUYG        10092750156         10.0 2024-04-30 04399860156          CT                  2390.63      00I29           TD24                           C. & T. s.r.l.      VIA FRANCESCO ALGAROTTI, 22  5.00000000 

In [36]:
clustered_fatture = dp.sequential_cluster(fatture_categorizzate, threshold=0.8)
reduced_row = dp.representatives(clustered_fatture)


Initial dataframe shape:
(25194, 16)
Dataframe shape after clustering:
(25194, 17)
Dataframe shape after selecting representatives:
(3988, 17)
Dataframe shape after clustering:
(25194, 17)
Dataframe shape after selecting representatives:
(3988, 17)


In [37]:
# Read CSV with semicolon separator since the data is semicolon-delimited
fornitori = pd.read_csv('categorizzazione_fornitori (2).csv', sep=';')
fornitori.loc[fornitori['CATEGORIA'] == 'Caff√®', 'CATEGORIA'] = 'Caffè'
print(f"Fornitori dataset shape: {fornitori.shape}")
print(f"Columns: {list(fornitori.columns)}")
print("\nSample of 20 rows:")
print(fornitori.sample(20).to_string(index=False))

Fornitori dataset shape: (1189, 5)
Columns: ['DENOMINAZIONE', 'P_IVA', 'MACRO_CATEGORIA', 'CATEGORIA', 'FISSO_VARIABILE_INVESTIMENTI']

Sample of 20 rows:
                          DENOMINAZIONE       P_IVA       MACRO_CATEGORIA             CATEGORIA FISSO_VARIABILE_INVESTIMENTI
                               MARR SPA 00000000000       Costi Variabili                  Food                    Variabile
    ELETTRICA BATTISTINI COMMERCITY SRL 00000000000     Altri Costi Fissi Manutenzione Generica                        Fisso
     MASCIARELLI Tenute Agricole s.r.l. 02442790693       Costi Variabili              Beverage                    Variabile
                      ITTICA URBANO SRL 10903181005       Costi Variabili                  Food                    Variabile
                            Thaler GmbH 00123080210       Costi Variabili              Beverage                    Variabile
SOCIET√Ä AGRICOLA TERRE FRIULANE S.R.L. 01916200304       Costi Variabili              Beverage

In [18]:
print("Unique categories in fornitori dataset:")
print("\nMACRO_CATEGORIA:")
print(fornitori['MACRO_CATEGORIA'].value_counts())
print("\nCATEGORIA:")
print(fornitori['CATEGORIA'].value_counts())
print("\nFISSO_VARIABILE_INVESTIMENTI:")
print(fornitori['FISSO_VARIABILE_INVESTIMENTI'].value_counts())

Unique categories in fornitori dataset:

MACRO_CATEGORIA:
MACRO_CATEGORIA
Costi Variabili          564
Altri Costi Fissi        410
Altri Costi Variabili    162
Investimenti              24
Utenze                    15
Affitto                    7
Costo Staff                3
Name: count, dtype: int64

CATEGORIA:
CATEGORIA
Beverage                     292
Food                         265
Manutenzione Generica         72
Materiali di Consumo          60
Altre Spese                   47
Altre Forniture               44
Allestimento eventi           38
Consulenza                    30
Attrezzature Cucina           28
Marketing                     27
Rappresentanza                26
Investimenti                  23
Spese Amministrative          21
Noleggi                       21
Abiti lavoro                  17
Software                      17
Allestimento Eventi           16
Manutenzione Impianti         14
Attrezzature Sala             10
Service                        8
Fotografo      

In [14]:
print("Unique categories in reduced_row dataset:")
print(reduced_row['CATEGORIA'].value_counts())

Unique categories in reduced_row dataset:
CATEGORIA
Food                             1635
Beverage                          960
Note                              400
Spese Amministrative               95
Attrezzature Sala                  89
Manutenzione Macchinari            81
Altre Forniture                    73
Spese Trasporto                    71
Materiali di Cucina                62
Materiali di consumo               50
Packaging                          49
Altre Spese                        44
Attrezzature Cucina                38
Spese Bancarie                     38
Manutenzione Impianti              36
Energia Elettrica                  35
Marketing                          28
Manutenzione Generica              26
Noleggi                            23
Software                           22
Abiti lavoro                       20
Lavanderia                         17
Service                            11
Investimenti                       11
Commissioni                        1

In [38]:

# Define a function to remove leading zeros and non-digit chars
import re

def normalize_piva(x):
    if pd.isna(x):
        return x
    s = str(x)
    # keep only digits
    digits = re.sub(r"\D+", "", s)
    # strip leading zeros
    return digits.lstrip('0') if digits != '' else digits

# Create normalized columns
reduced_row['piva_norm'] = reduced_row['P_IVA'].astype(str).apply(normalize_piva)
fornitori['piva_norm'] = fornitori['P_IVA'].astype(str).apply(normalize_piva)

# Merge on the normalized column
merged_data = reduced_row.merge(
    fornitori[['piva_norm', 'CATEGORIA']],
    left_on='piva_norm',
    right_on='piva_norm',
    how='left',
    suffixes=('', '_FORNITORE')
)

# Drop temporary columns
merged_data.drop(columns=['piva_norm'], inplace=True)

print(f"Merged dataset shape: {merged_data.shape}")
print(f"Number of rows with matched supplier category: {merged_data['CATEGORIA_FORNITORE'].notna().sum()}")
print(f"Number of rows without matched supplier category: {merged_data['CATEGORIA_FORNITORE'].isna().sum()}")

# Show sample of matched rows and unmatched rows for inspection
print("\nSample matched rows:")
print(merged_data[merged_data['CATEGORIA_FORNITORE'].notna()].sample(1000).to_string(index=False))
print("\nSample unmatched rows:")
print(merged_data[merged_data['CATEGORIA_FORNITORE'].isna()].head(10).to_string(index=False))

Merged dataset shape: (3988, 18)
Number of rows with matched supplier category: 3988
Number of rows without matched supplier category: 0

Sample matched rows:
                      SourceName CODICE_COMMITTENTE ALIQUOTA_IVA       DATA       P_IVA UNITAMISURA IMPORTO_TOTALE_DOCUMENTO     NUMERO TIPO_DOCUMENTO                                                             RAGIONE_SOCIALE                                   SEDE_INDIRIZZO_CESSIONARIO       QUANTITA                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           

In [44]:
columns_to_keep = ['P_IVA', 'RAGIONE_SOCIALE', 'DESCRIZIONE','IMPONIBILE_IMPORTO', 'PREZZO_UNITARIO', 'CATEGORIA', 'cluster', 'CATEGORIA_FORNITORE']
minimal_columns = merged_data[columns_to_keep]
cat_matched = minimal_columns[minimal_columns['CATEGORIA_FORNITORE'] == minimal_columns['CATEGORIA']]
not_matched = minimal_columns[minimal_columns['CATEGORIA_FORNITORE'] != minimal_columns['CATEGORIA']]
print(not_matched)
not_matched.to_csv('not_matched.csv', index=False)

            P_IVA                             RAGIONE_SOCIALE  \
39    00059380055  DISTILLERIA CANELLESE C. BOCCHINO & C. SRL   
40    00072692443                       PUERTA DIEGO MANSILLA   
42    00078410057                     COPPO S.I.V.A.S. S.R.L.   
44    00088470521           CASTELLO DI BOSSI SOC. AGRIC. SRL   
45    00088470521           CASTELLO DI BOSSI SOC. AGRIC. SRL   
...           ...                                         ...   
3981  15844561009                         Enel Energia S.p.A.   
3982  15844561009                         Enel Energia S.p.A.   
3985  16714991003                            WALLY JOB S.R.L.   
3986  16714991003                            WALLY JOB S.R.L.   
3987  16714991003                            WALLY JOB S.R.L.   

                                            DESCRIZIONE IMPONIBILE_IMPORTO  \
39                                IMPOSTA FABBRICAZIONE              29.46   
40               Aliq. Iva 22% Integrazione iva al  22%        

In [ ]:
fatture_categorizzate = pd.read_csv('categorized_output.csv', dtype=str)
test_df = fatture_categorizzate.sample(100)
test_clustered = dp.sequential_cluster(test_df, threshold=0.8)
test_reduced = dp.representatives(test_clustered)
